In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind


# Merged file :


A detailed trace of client interactions online, divided into two parts: pt_1 and pt_2. 

In [2]:
file_path = r'C:\Users\ALEV PC\Desktop\Project\Data\Raw data\df_final_web_data_pt_1.txt'
df1 = pd.read_csv(file_path)
df1.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [3]:
file_path = r'C:\Users\ALEV PC\Desktop\Project\Data\Raw data\df_final_web_data_pt_2.txt'
df2 = pd.read_csv(file_path)
df2.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58


In [4]:
df3 = pd.concat([df1, df2], axis=0)
df3.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [5]:
file_path = r'C:\Users\ALEV PC\Desktop\Project\Data\Raw data\df_final_experiment_clients.txt'
df_groups = pd.read_csv(file_path)
df_groups.head()

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


In [6]:
df = pd.merge(df3, df_groups, on='client_id', how='inner')
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test


# Time Spent in Each Step for all users

In [7]:
import pandas as pd

# Convert 'date_time' column to datetime type
df['date_time'] = pd.to_datetime(df['date_time'])

# Sort by - "client_id", "visitor_id", "visit_id", "date_time"
df_sorted = df.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time']).copy()
number_of_clients_without_group = df_sorted[df_sorted["Variation"].isna() == True]["client_id"].nunique()
df_sorted = df_sorted[df_sorted["Variation"].isna() == False]
#display(df_sorted.head())

# Calculation
df_sorted['end_of_the_step'] = df_sorted.groupby('visit_id')['date_time'].shift(-1)
#display(df_sorted.head(7))

# Time difference
df_sorted['time_diff'] = (df_sorted['end_of_the_step'] - df_sorted['date_time']).dt.round('s')
#display(df_sorted)
# Average per step
#df_sorted.groupby(['visit_id', 'process_step'])[['time_diff']].mean().rename(columns={'time_diff': 'average'})
time_spent_all_users = df_sorted.groupby(['visit_id', 'process_step','Variation'])[['time_diff']].mean().rename(columns={'time_diff': 'average'})
time_spent_all_users = time_spent_all_users.groupby(['Variation','process_step'])[['average']].mean()

custom_dict = {"start":0, "step_1": 1, "step_2": 2, "step_3": 3, "confirmation": 4}

time_spent_all_users = time_spent_all_users.sort_values(by=["Variation","process_step"], key=lambda x: x.map(custom_dict)).reset_index()
display(time_spent_all_users)
time_spent_all_users.to_csv("Time_spent_all_users.csv", index=False)

,Variation,process_step,average
0,Control,start,0 days 00:00:50.314018583
1,Test,start,0 days 00:00:47.079617575
2,Control,step_1,0 days 00:00:45.722422076
3,Test,step_1,0 days 00:00:52.446021123
4,Control,step_2,0 days 00:01:37.066683737
5,Test,step_2,0 days 00:01:31.150916169
6,Control,step_3,0 days 00:02:16.204325116
7,Test,step_3,0 days 00:01:59.985044564
8,Control,confirm,0 days 00:02:11.088096003
9,Test,confirm,0 days 00:03:15.091538918


In [8]:
df_sorted.head(20)

# df_sorted.to_csv("df_sorted.csv", index=False)


,client_id,visitor_id,visit_id,process_step,date_time,Variation,end_of_the_step,time_diff
70803,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,Test,2017-04-15 12:58:03,0 days 00:00:07
70802,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,Test,2017-04-15 12:58:35,0 days 00:00:32
70801,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,Test,2017-04-15 13:00:14,0 days 00:01:39
70800,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,Test,2017-04-15 13:00:34,0 days 00:00:20
70799,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,Test,NaT,NaT
135918,647,66758770_53988066587,40369564_40101682850_311847,start,2017-04-12 15:41:28,Test,2017-04-12 15:41:35,0 days 00:00:07
135917,647,66758770_53988066587,40369564_40101682850_311847,step_1,2017-04-12 15:41:35,Test,2017-04-12 15:41:53,0 days 00:00:18
135916,647,66758770_53988066587,40369564_40101682850_311847,step_2,2017-04-12 15:41:53,Test,2017-04-12 15:45:02,0 days 00:03:09
135915,647,66758770_53988066587,40369564_40101682850_311847,step_3,2017-04-12 15:45:02,Test,2017-04-12 15:47:45,0 days 00:02:43
135912,647,66758770_53988066587,40369564_40101682850_311847,confirm,2017-04-12 15:47:45,Test,NaT,NaT


In [9]:
# df_sorted.to_csv("df_sorted.csv", index=False)

In [ ]:
# Save the average time differences to a CSV file
avg_time_diffs.to_csv('average_time_differences.csv')

print("CSV file saved successfully.")


# Independent Samples T-test on Time spent on each step

In [1]:
import numpy as np
from scipy.stats import ttest_ind

# Data
control_times = {
    'start': 50,
    'step 1': 45,
    'step 2': 97,
    'step 3': 136,
    'confirm': 131
}

test_times = {
    'start': 47,
    'step 1': 52,
    'step 2': 91,
    'step 3': 119,
    'confirm': 195
}

# Perform t-test for each process step
for step in control_times.keys():
    control_time = control_times[step]
    test_time = test_times[step]
    
    # Perform t-test
    t_statistic, p_value = ttest_ind([control_time], [test_time], equal_var=False)
    
    # Print results
    print(f"Process Step: {step}")
    print(f"Control Time: {control_time}")
    print(f"Test Time: {test_time}")
    print(f"T-statistic: {t_statistic}")
    print(f"P-value: {p_value}")
    if p_value < 0.05:
        print("Result: Reject Null Hypothesis - Significant Difference")
    else:
        print("Result: Fail to Reject Null Hypothesis - No Significant Difference")
    print("="*50)


Process Step: start
Control Time: 50
Test Time: 47
T-statistic: nan
P-value: nan
Result: Fail to Reject Null Hypothesis - No Significant Difference
Process Step: step 1
Control Time: 45
Test Time: 52
T-statistic: nan
P-value: nan
Result: Fail to Reject Null Hypothesis - No Significant Difference
Process Step: step 2
Control Time: 97
Test Time: 91
T-statistic: nan
P-value: nan
Result: Fail to Reject Null Hypothesis - No Significant Difference
Process Step: step 3
Control Time: 136
Test Time: 119
T-statistic: nan
P-value: nan
Result: Fail to Reject Null Hypothesis - No Significant Difference
Process Step: confirm
Control Time: 131
Test Time: 195
T-statistic: nan
P-value: nan
Result: Fail to Reject Null Hypothesis - No Significant Difference


C:\Users\ALEV PC\Desktop\Project\venv\Lib\site-packages\scipy\stats\_stats_py.py:1081: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\ALEV PC\Desktop\Project\venv\Lib\site-packages\scipy\stats\_stats_py.py:1081: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


# Mann-Whitney U-test and the Kruskal-Wallis

In [3]:
from scipy.stats import mannwhitneyu, kruskal

# Data
control_times = [50, 45, 97, 136, 131]
test_times = [47, 52, 91, 119, 195]

# Mann-Whitney U-test
u_statistic, p_value_mannwhitney = mannwhitneyu(control_times, test_times)
print("Mann-Whitney U-test:")
print(f"U-statistic: {u_statistic}")
print(f"P-value: {p_value_mannwhitney}")
if p_value_mannwhitney < 0.05:
    print("Result: Reject Null Hypothesis - Significant Difference")
else:
    print("Result: Fail to Reject Null Hypothesis - No Significant Difference")
print("="*50)

# Kruskal-Wallis test
_, p_value_kruskal = kruskal(control_times, test_times)
print("Kruskal-Wallis Test:")
print(f"P-value: {p_value_kruskal}")
if p_value_kruskal < 0.05:
    print("Result: Reject Null Hypothesis - Significant Difference")
else:
    print("Result: Fail to Reject Null Hypothesis - No Significant Difference")


Mann-Whitney U-test:
U-statistic: 12.0
P-value: 1.0
Result: Fail to Reject Null Hypothesis - No Significant Difference
Kruskal-Wallis Test:
P-value: 0.9168149485280722
Result: Fail to Reject Null Hypothesis - No Significant Difference


# Split the df by Variation : Test and Control Group

In [ ]:
import pandas as pd

# Split the DataFrame into Test and Control groups based on the "Variation" column
test_group = df_sorted[df_sorted['Variation'] == 'Test']
control_group = df_sorted[df_sorted['Variation'] == 'Control']

# Save Test group to CSV
test_group.to_csv('test_group.csv', index=False)

# Save Control group to CSV
control_group.to_csv('control_group.csv', index=False)


# Completion Rate for all Users - Exluding error rates

In [8]:
import pandas as pd

# Filter for rows where the process_step is "confirm"
confirm_df = df_sorted[df_sorted['process_step'] == 'confirm']

# Group by 'client_id' and select the row with the maximum date_time for each client_id
last_confirm_rows = confirm_df.groupby('client_id').apply(lambda x: x.loc[x['date_time'].idxmax()])

# Display the result
last_confirm_rows


C:\Users\ALEV PC\AppData\Local\Temp\ipykernel_14736\3180173949.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_confirm_rows = confirm_df.groupby('client_id').apply(lambda x: x.loc[x['date_time'].idxmax()])


,client_id,visitor_id,visit_id,process_step,date_time,Variation,end_of_the_step,time_diff
client_id,,,,,,,,
555,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,Test,NaT,NaT
647,647,66758770_53988066587,40369564_40101682850_311847,confirm,2017-04-12 15:47:45,Test,NaT,NaT
1195,1195,766842522_69992551638,393817425_39015278493_996341,confirm,2017-04-05 20:19:31,Control,NaT,NaT
1197,1197,753759429_54481946928,71862471_21202285428_848395,confirm,2017-04-01 17:17:59,Control,NaT,NaT
1336,1336,920624746_32603333901,614001770_19101025926_112779,confirm,2017-05-08 08:23:00,Test,NaT,NaT
...,...,...,...,...,...,...,...,...
9998851,9998851,660529049_81285947655,310744937_17135608286_213964,confirm,2017-04-05 12:08:43,Test,NaT,NaT
9998964,9998964,502193753_63097042630,699981922_52880598802_858635,confirm,2017-04-05 23:50:02,Test,NaT,NaT
9999009,9999009,212849256_28644888524,149965839_24823291142_245569,confirm,2017-03-31 07:20:36,Test,NaT,NaT


In [9]:
# Count the occurrences of test and control variations
variation_counts = last_confirm_rows['Variation'].value_counts()

# Display the counts
print("Count of Test variation:", variation_counts.get('Test', 0))
print("Count of Control variation:", variation_counts.get('Control', 0))


Count of Test variation: 18687
Count of Control variation: 15434


In [ ]:
# Save the DataFrame as a CSV file
last_confirm_rows.to_csv('last_confirm_rows.csv', index=False)

print("CSV file created successfully: last_confirm_rows.csv")


In [10]:
import pandas as pd

# Calculate the total number of users and the number of "confirm" users
total_users = len(df['client_id'].unique())
confirm_users = len(df[df['process_step'] == 'confirm']['client_id'].unique())

# Calculate the proportion of "confirm" users compared to all the users
confirm_proportion = confirm_users / total_users

# Calculate confirm rates for "Test" and "Control" groups separately
test_confirm_rate = len(df[(df['Variation'] == 'Test') & (df['process_step'] == 'confirm')]['client_id'].unique()) / len(df[df['Variation'] == 'Test']['client_id'].unique())
control_confirm_rate = len(df[(df['Variation'] == 'Control') & (df['process_step'] == 'confirm')]['client_id'].unique()) / len(df[df['Variation'] == 'Control']['client_id'].unique())

# Display the results
print(f"Proportion of 'confirm' users compared to all users: {confirm_proportion:.2%}")
print(f"Confirm rate for 'Test' group: {test_confirm_rate:.2%}")
print(f"Confirm rate for 'Control' group: {control_confirm_rate:.2%}")


Proportion of 'confirm' users compared to all users: 67.70%
Confirm rate for 'Test' group: 69.29%
Confirm rate for 'Control' group: 65.59%


# Mann-Whitney U Test (Wilcoxon rank-sum test) and Kruskal-Wallis Test for Time Spent on Each Process


In [ ]:
Mann-Whitney U Test (Wilcoxon rank-sum test):

Assumption: The samples are independent and come from populations with the same shape.
Null Hypothesis (H0): The distributions of both groups are equal.
Alternative Hypothesis (H1): The distributions of both groups are not equal.
Interpretation: If the p-value is less than the significance level (e.g., 0.05), 
we reject the null hypothesis and conclude that there is a significant difference between the distributions of the two groups.

Kruskal-Wallis Test:

Assumption: The samples are independent and come from populations with the same shape.
Null Hypothesis (H0): The distributions of all groups are equal.
Alternative Hypothesis (H1): At least one group's distribution is different from the others.
Interpretation: If the p-value is less than the significance level (e.g., 0.05), we reject the null hypothesis and conclude that there is a significant difference between the distributions of the groups.

In [11]:
import scipy.stats as stats

# Assuming you have two groups: 'Test' and 'Control'
group_test = df_sorted[df_sorted['Variation'] == 'Test']
group_control = df_sorted[df_sorted['Variation'] == 'Control']

# Perform the Mann-Whitney U test
mwu_stat, mwu_pvalue = stats.mannwhitneyu(group_test['time_diff'].dropna(), group_control['time_diff'].dropna())

# Perform the Kruskal-Wallis test
kw_stat, kw_pvalue = stats.kruskal(group_test['time_diff'].dropna(), group_control['time_diff'].dropna())

# Display the results
print("Mann-Whitney U test p-value:", mwu_pvalue)
print("Kruskal-Wallis test p-value:", kw_pvalue)


Mann-Whitney U test p-value: 0.3169330407444608
Kruskal-Wallis test p-value: 0.31693302738687823


# Two-sample t-test - Error Rates

In [13]:
import scipy.stats as stats

# Define error rates for control and test variations
control_error_rates = [37.14, 29.77, 12.77, 12.48, 7.85]
test_error_rates = [31.06, 34.56, 15.14, 10.12, 9.13]

# Perform two-sample t-test
t_stat, t_pvalue = stats.ttest_ind(control_error_rates, test_error_rates)

# Display the results
print("t-statistic:", t_stat)
print("p-value:", t_pvalue)

t-statistic: -4.549928160846744e-16
p-value: 0.9999999999999997


In [ ]:
# Null hypothesis : 

In [12]:
from scipy.stats import ttest_ind

# We have 'Test' and 'Control' variation groups
group_test = df_sorted[df_sorted['Variation'] == 'Test']
group_control = df_sorted[df_sorted['Variation'] == 'Control']

# Convert timedelta to numerical values (e.g., seconds) for each group
data_points_group1 = group_test['time_diff'].dropna().dt.total_seconds()
data_points_group2 = group_control['time_diff'].dropna().dt.total_seconds()

# Perform the two-sample t-test
t_stat, t_pvalue = ttest_ind(data_points_group1, data_points_group2)

# Display the results
print("Two-sample t-test p-value:", t_pvalue)



Two-sample t-test p-value: 0.005741421428463605


# C-square test to compare error rates

In [24]:
# There is no difference in error rates between the 'Test' and 'Control' groups

In [26]:

# A p-value of 1.0 suggests that there is no significant difference in error rates between the two groups. 
# In other words, the null hypothesis, which states that there is no difference in error rates between the 'Test' and 'Control' groups, cannot be rejected. 
# Therefore, based on the chi-square test, there is no evidence to suggest that the error rates differ between the two groups.

# C-square test to compare completion rates


In [27]:
# Null Hypothesis : There is no difference in completion rates between the 'Test' and 'Control' groups.
# Alternative Hypothesis : There is a difference in completion rates between the 'Test' and 'Control' groups.

In [28]:
import pandas as pd

# Assuming df is your DataFrame containing the data
# Sort the data by client_id, visit_id, and date_time
df_sorted = df.sort_values(by=['client_id', 'visit_id', 'date_time'])
df_sorted.head()
# Filter out errors where clients go back to previous steps

#valid_rows = df_sorted.groupby(['client_id']).apply(lambda x: x.iloc[-1])
#valid_rows = valid_rows[valid_rows['process_step'] == 'confirm']

# Calculate completion rates for each group
#test_group = valid_rows[valid_rows['Variation'] == 'Test']
#control_group = valid_rows[valid_rows['Variation'] == 'Control']

#test_completion_rate = len(test_group) / len(valid_rows)
#control_completion_rate = len(control_group) / len(valid_rows)


# Display the results
#print("Test Completion Rate:", test_completion_rate)
#print("Control Completion Rate:", control_completion_rate)


,client_id,visitor_id,visit_id,process_step,date_time,Variation
280826,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36,NaN
280825,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45,NaN
280824,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31,NaN
280823,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05,NaN
280822,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09,NaN


In [ ]:
# A small p-value (typically less than 0.05) indicates strong evidence against the null hypothesis, suggesting that there is 
# a significant difference in completion rates between the 'Test' and 'Control' groups. 
# In other words, the completion rates are significantly different between the two groups.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Observed and Expected Completion Rates
observed_completion_rates = [0.2313628259416091, 0.17660742143971472]
expected_completion_rates = [0.2, 0.2]  # Assuming expected completion rate is 0.2 for both groups

# Data
groups = ['Test', 'Control']
observed = np.array(observed_completion_rates)
expected = np.array(expected_completion_rates)

# Create bar plot
fig, ax = plt.subplots(figsize=(8, 6))
bar_width = 0.35
index = np.arange(len(groups))

bar1 = ax.bar(index, observed, bar_width, label='Observed', color='blue')
bar2 = ax.bar(index + bar_width, expected, bar_width, label='Expected', color='orange')

# Add labels, title, and legend
ax.set_xlabel('Groups')
ax.set_ylabel('Completion Rate')
ax.set_title('Observed vs Expected Completion Rates')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(groups)
ax.legend()

# Show plot
plt.show()

